<a href="https://colab.research.google.com/github/aajunior43/notebooks-google-colab/blob/main/Ol%C3%A1%2C_este_%C3%A9_o_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Importando as bibliotecas necessárias
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
import zipfile
from google.colab import files
import shutil

def download_images_from_url(url):
    # Enviando uma solicitação GET para a URL
    response = requests.get(url)

    # Verificando se a solicitação foi bem-sucedida
    if response.status_code == 200:
        # Usando BeautifulSoup para analisar o conteúdo da página
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando todas as tags de imagem
        img_tags = soup.find_all('img')

        # Lista para armazenar as imagens baixadas
        imagens = []

        # Loop através de todas as tags de imagem e baixando cada imagem
        for img_tag in img_tags:
            img_url = img_tag.get('src')
            if img_url:
                # Verificando se a URL da imagem é relativa e convertendo-a em absoluta, se necessário
                img_url = urljoin(url, img_url)

                # Enviando uma solicitação GET para a URL da imagem
                img_response = requests.get(img_url)

                # Verificando se a solicitação foi bem-sucedida e se a imagem é do tipo desejado
                if img_response.status_code == 200 and img_url.lower().endswith(('.jpeg', '.jpg', '.png')):
                    # Salvando a imagem na lista
                    imagens.append((img_url, img_response.content))

        # Ordenando as imagens do maior para o menor tamanho
        imagens.sort(key=lambda x: len(x[1]), reverse=True)

        # Salvando as imagens na pasta 'imagens' e renomeando conforme o tamanho
        for i, (img_url, img_content) in enumerate(imagens, start=1):
            with open(f'imagens/{i}.{img_url.split(".")[-1]}', 'wb') as img_file:
                img_file.write(img_content)
            print(f'Imagem {i} baixada com sucesso de {url}!')
    else:
        print(f'Falha ao acessar a URL {url}')

# Solicitando as URLs do usuário
urls = []
while True:
    url = input("Por favor, insira a URL da página que você deseja raspar (ou 'sair' para terminar): ").strip()
    if url.lower() == 'sair':
        break
    urls.append(url)

# Criando uma pasta para salvar as imagens
os.makedirs('imagens', exist_ok=True)

# Loop através de todas as URLs fornecidas e baixando as imagens
for url in urls:
    download_images_from_url(url.strip())

# Criando um arquivo ZIP com todas as imagens
with zipfile.ZipFile('imagens.zip', 'w') as zipf:
    for root, _, files_in_folder in os.walk('imagens'):
        for file in files_in_folder:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), 'imagens'))

# Baixando o arquivo ZIP para a máquina local
files.download('imagens.zip')

# Removendo a pasta 'imagens' e todas as imagens dentro dela
shutil.rmtree('imagens')


KeyboardInterrupt: ignored